# Introducing Keras

Let's use Keras on the MNIST data set again, this time using a Convolutional Neural Network that's better suited for image processing. CNN's are less sensitive to where in the image the pattern is that we're looking for.

With a multi-layer perceptron, we achieved around 97% accuracy. Let's see if we can beat that.

As before we'll start by importing the stuff we need, including the new layer types we talked about:

In [3]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten, Activation
from keras.optimizers import RMSprop

Using TensorFlow backend.


Raw Data Set

In [4]:
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K

#Image dim
img_width, img_height = 100, 100

#Parameters
train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 5000
nb_validation_samples = 2000
# epochs = 50
# batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

11493376/11490434 [==============================] - 97s 8us/step


We need to shape the data differently then before. Since we're treating the data as 2D images of 28x28 pixels instead of a flattened stream of 784 pixels, we need to shape it accordingly. Depending on the data format Keras is set up for, this may be 1x28x28 or 28x28x1 (the "1" indicates a single color channel, as this is just grayscale. If we were dealing with color images, it would be 3 instead of 1 since we'd have red, green, and blue color channels)

In [5]:
#from keras import backend as K

# if K.image_data_format() == 'channels_first':
#     train_images = mnist_train_images.reshape(mnist_train_images.shape[0], 1, 28, 28)
#     test_images = mnist_test_images.reshape(mnist_test_images.shape[0], 1, 28, 28)
#     input_shape = (1, 28, 28)
# else:
#     train_images = mnist_train_images.reshape(mnist_train_images.shape[0], 28, 28, 1)
#     test_images = mnist_test_images.reshape(mnist_test_images.shape[0], 28, 28, 1)
#     input_shape = (28, 28, 1)
    
# train_images = train_images.astype('float32')
# test_images = test_images.astype('float32')
# train_images /= 255
# test_images /= 255

As before we need to convert our train and test labels to be categorical in one-hot format:

In [6]:
train_labels = keras.utils.to_categorical(mnist_train_labels, 10)
test_labels = keras.utils.to_categorical(mnist_test_labels, 10)

One of the training images with its label:

In [7]:
# import matplotlib.pyplot as plt

# def display_sample(num):
#     #Print the one-hot array of this sample's label 
#     print(train_labels[num])  
#     #Print the label converted back to a number
#     label = train_labels[num].argmax(axis=0)
#     #Reshape the 768 values to a 28x28 image
#     image = train_images[num].reshape([28,28])
#     plt.title('Sample: %d  Label: %d' % (num, label))
#     plt.imshow(image, cmap=plt.get_cmap('gray_r'))
#     plt.show()
    
# display_sample(1234)

[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]


<Figure size 640x480 with 1 Axes>

Now for the meat of the problem. Setting up a convolutional neural network involves more layers. Not all of these are strictly necessary; you could run without pooling and dropout, but those extra steps help avoid overfitting and help things run faster.

We'll start with a 2D convolution of the image - it's set up to take 32 windows, or "filters", of each image, each filter being 3x3 in size.

We then run a second convolution on top of that with 64 3x3 windows - this topology is just what comes recommended within Keras's own examples. Again you want to re-use previous research whenever possible while tuning CNN's, as it is hard to do.

Next we apply a MaxPooling2D layer that takes the maximum of each 2x2 result to distill the results down into something more manageable.

A dropout filter is then applied to prevent overfitting.

Next we flatten the 2D layer we have at this stage into a 1D layer. So at this point we can just pretend we have a traditional multi-layer perceptron...

... and feed that into a hidden, flat layer of 128 units.

We then apply dropout again to further prevent overfitting.

And finally, we feed that into our final 10 units where softmax is applied to choose our category of 0-9.

In [8]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
# 64 3x3 kernels
model.add(Conv2D(64, (3, 3), activation='relu'))
# Reduce by taking the max of each 2x2 block
model.add(MaxPooling2D(pool_size=(2, 2)))
# Dropout to avoid overfitting
model.add(Dropout(0.25))
# Flatten the results to one dimension for passing into our final layer
model.add(Flatten())
# A hidden layer to learn with
model.add(Dense(128, activation='relu'))
# Another dropout
model.add(Dropout(0.5))
# Final categorization from 0-9 with softmax
model.add(Dense(10, activation='softmax'))

Model Description:

In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________

We are still doing multiple categorization, so categorical_crossentropy is still the right loss function to use. We'll use the Adam optimizer, although the example provided with Keras uses RMSProp. You might want to try both if you have time.

In [10]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

#Train - batches of 32

In [11]:
history = model.fit(train_images, train_labels,
                    batch_size=32,
                    epochs=1,
                    verbose=2,
                    validation_data=(test_images, test_labels))

Train on 60000 samples, validate on 10000 samples
Epoch 1/1
 - 342s - loss: 0.1867 - acc: 0.9447 - val_loss: 0.0508 - val_acc: 0.9833


Evaluate

In [12]:
score = model.evaluate(test_images, test_labels, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.05084224361399538
Test accuracy: 0.9833


In [13]:
model.save_weights('first_try.h5')